In [2]:
#installing all libraries
#keras for modelling
import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
#using for visualization
import matplotlib.pyplot as plt
#using opencv and os for streaming and reading images from directory
import cv2 as cv
import cv2,os

Using TensorFlow backend.
C:\Users\haris\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\haris\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\haris\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\haris\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
data_path='E:/Public Dataset/Mask and No mask'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]
#Creating empty dictionary and check the number of folders
label_dict=dict(zip(categories,labels)) 

print(label_dict)
print(categories)
print(labels)

{'Goggles': 0, 'without_mask': 1, 'with_mask': 2}
['Goggles', 'without_mask', 'with_mask']
[0, 1, 2]


In [4]:
img_size=224
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)


In [5]:
import numpy as np
from keras.utils import np_utils
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [6]:
np.save('data',data)
np.save('target',new_target)


In [7]:
#loading data in array form
import numpy as np
data=np.load('data.npy')
target=np.load('target.npy')

In [8]:
#Giving image size and RGB
IMAGE_SIZE = [224, 224]
#Modelling 
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
 # useful for getting number of classes
#folders = glob('E:/Public Dataset/Mask and No mask')
  

# our layers - you can add more if you want
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)
prediction = Dense(3, activation='softmax')(x) #we have three outputs(multinomial classification)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# Telling the model what cost funtion , optimizer to use
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [9]:
#Creating training and test datset
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [ ]:
#training our model
Model_history=model.fit(train_data,train_target,epochs=2,validation_split=0.2)


Train on 1448 samples, validate on 362 samples
Epoch 1/2
  64/1448 [>.............................] - ETA: 9:30 - loss: 5.6962 - accuracy: 0.4219

KeyboardInterrupt: 

In [ ]:
#saving the model to load 
#model.save('VGG16.h5')

In [13]:
import cv2 as cv
from keras.models import load_model

#loading our saved model
model = load_model('VGG16.h5')

#using Intel Haarcascade classifier to detect  face 
face_clsfr=cv.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv.VideoCapture(0)

labels_dict={0:'Goggles',1:'without_mask',2:'with_mask'} #defing labels
color_dict={0:(0,255,0),1:(0,0,255),2:(255, 0, 0)} #Different color for box
while(True):

    ret,img=source.read()
    #gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=img[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        result=model.predict(reshaped)
        print(result)
        label=np.argmax(result,axis=1)[0]
      
        cv.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv.imshow('LIVE',img)
    key=cv.waitKey(1)
    
    if(key==27):
        break
        
cv.destroyAllWindows()
source.release()

[[3.1678102e-04 9.6003550e-01 3.9647661e-02]]
[[0.00142243 0.99454945 0.00402812]]
[[0.00622283 0.98540145 0.00837576]]
[[0.00448348 0.98540676 0.01010981]]
[[0.00667237 0.92992944 0.06339827]]
[[0.00659617 0.93024904 0.06315473]]
[[0.00691058 0.9319294  0.0611601 ]]
[[0.00359628 0.9589148  0.03748893]]
[[5.5645425e-05 9.9431324e-01 5.6312419e-03]]
[[0.01294085 0.8999908  0.08706832]]
[[0.00822559 0.9092764  0.082498  ]]


In [10]:
#importing nececssary library
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# loading images the function to be in right shape shape
def load_image(filename):
    # Load the image
    img = load_img(filename, target_size=(224,224))
    # Convert the image to array
    img = img_to_array(img)
    # Reshape the image into a sample of 1 channel
    img = img.reshape(1, 224, 224, 3)
    # Prepare it as pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img

# Load an image and predict the apparel class
labels_dict={0:'Goggles',1:'without_mask',2:'with_mask'}
img = load_image('checkimageofyourchoice.jpg')
# Load the saved model for prediction
model = load_model('VGG16.h5')
# Predict for particular class 
class_prediction = model.predict(img)
label=np.argmax(class_prediction,axis=1)[0]

print(labels_dict[label])


without_mask


with_mask
